In [1125]:
#aim of this script is to transform vague country names to names found in the iso3166 standard country list
###iterate through all the country/region values
###check if the value can be matched with a value in the iso3166 options
###if the there's a match, add the value from the iso3166 class onto a new column called 'std_country/region', create another column which indicates if a match was found 1, 0
###if there's no match update the row with 0 on the new column
import pandas as pd
from iso3166 import countries
#from difflib import SequenceMatcher
from datetime import datetime

In [1126]:
start_time = datetime.now()
batch_size = 10
match_count = 0
iso_countries_df = pd.DataFrame() 
hs_countries_df = pd.DataFrame()
countries_wo_match = pd.DataFrame()
audit_frame = pd.DataFrame()
with open('contact-country-region-values.csv') as f:
    file = f.readlines()
    row_count = len(file)

In [1127]:
def match_counter():
    global match_count
    match_count = match_count + 1
    return match_count

In [1128]:
def match_maker(csv_chunk):
    global csv_data
    global hs_countries_df
    csv_data = csv_chunk.dropna(subset=['Country/Region'])
    hs_countries_df = hs_countries_df.append(csv_data)
    for idx, country_name in csv_data['Country/Region'].iteritems():
        print('Converting contact {} out of {}.'.format(match_counter(), row_count), 'Runtime duration {}'.format(datetime.now() - start_time))
        try:
            new_country_name = countries.get(country_name)
            hs_countries_df.loc[idx, 'new_country_name'] = new_country_name[0]
            hs_countries_df.loc[idx, 'Matched'] = True
        except:
            #for std_cname in iso_countries_df['name'].iteritems():
                #new_old_simil = similar(country_name.lower(), std_cname[1].lower())
                #if new_old_simil > 0.5:
                #    hs_countries_df.loc[idx, 'new_country_name'] = std_cname[1]
                #    hs_countries_df.loc[idx, 'similarity_score'] = new_old_simil
                #else:
            hs_countries_df.loc[idx, 'Matched'] = False

In [1129]:
#check for the similarity of the strings based on the levenshtein distance
#def similar(a, b):
#    return SequenceMatcher(None, a, b).ratio()

In [1118]:
#import the hubspot csv export into a dataframe
#import the countries functions data into a dataframe for mathcing purposes
#for iso_data in countries:
#        iso_countries_df = iso_countries_df.append({'name':iso_data[0],
#                                            'alpha2_ccode':iso_data[1],
#                                            'alpha3_code':iso_data[2],
#                                            'numeric':iso_data[3]},
#                                            ignore_index=True)

for chunk in pd.read_csv('contact-country-region-values.csv', chunksize=batch_size, nrows=row_count):
    match_maker(chunk)

print('Dataframe dimensions {}'.format(hs_countries_df.shape))

Converting contact 1 out of 2108571. Runtime duration 0:00:09.946335
Converting contact 2 out of 2108571. Runtime duration 0:00:09.952783
Converting contact 3 out of 2108571. Runtime duration 0:00:09.961854
Converting contact 4 out of 2108571. Runtime duration 0:00:09.963939
Converting contact 5 out of 2108571. Runtime duration 0:00:09.966952
Converting contact 6 out of 2108571. Runtime duration 0:00:09.969754
Converting contact 7 out of 2108571. Runtime duration 0:00:10.035862
Converting contact 8 out of 2108571. Runtime duration 0:00:10.043856
Converting contact 9 out of 2108571. Runtime duration 0:00:10.049603
Converting contact 10 out of 2108571. Runtime duration 0:00:10.054193
Converting contact 11 out of 2108571. Runtime duration 0:00:10.058305
Converting contact 12 out of 2108571. Runtime duration 0:00:10.061399
Converting contact 13 out of 2108571. Runtime duration 0:00:10.062990
Converting contact 14 out of 2108571. Runtime duration 0:00:10.167065
Converting contact 15 out of 

In [1119]:
print(hs_countries_df['Matched'].value_counts())
print(hs_countries_df.columns)

False    53
True     32
Name: Matched, dtype: int64
Index(['Associated Company', 'Associated Company ID',
       'Became a Marketing Qualified Lead Date', 'Contact ID',
       'Contact country', 'Country Code', 'Country of Respondents',
       'Country/Region', 'Create Date', 'Email', 'First Name', 'HQ Country',
       'IP Country', 'IP Country Code', 'Inferred Country', 'Last Name',
       'Lead Source', 'Matched', 'Most Recent Lead Source', 'Original Source',
       'Original Source Drill-Down 1', 'Original Source Drill-Down 2',
       'Product Code', 'Salesforce Contact ID', 'Salesforce Lead ID',
       'new_country_name'],
      dtype='object')


In [1120]:
no_match_ls = hs_countries_df.loc[hs_countries_df['Matched'] == False, 'Lead Source']
print(no_match_ls.value_counts())

Sales Created     51
Contact Sales      1
Account Signup     1
Name: Lead Source, dtype: int64


In [1121]:
no_match_countries = hs_countries_df.loc[hs_countries_df['Matched'] == False, 'Country/Region']
print(no_match_countries.value_counts())

United States     39
United Kingdom    13
united states      1
Name: Country/Region, dtype: int64


In [1122]:
print(hs_countries_df[['Matched','Country/Region','new_country_name']].head())

   Matched Country/Region new_country_name
0    False  united states              NaN
1     True        myanmar          Myanmar
2     True       thailand         Thailand
6     True          india            India
7     True          india            India


In [1123]:
print(hs_countries_df.head())
print(hs_countries_df.tail())

  Associated Company  Associated Company ID  \
0                NaN                    NaN   
1                NaN                    NaN   
2                NaN                    NaN   
6                NaN                    NaN   
7                NaN                    NaN   

  Became a Marketing Qualified Lead Date  Contact ID Contact country  \
0                                    NaN   311098494   United States   
1                                    NaN   311097942         Myanmar   
2                                    NaN   311088532        Thailand   
6                                    NaN   311094170           India   
7                                    NaN   310420803           India   

   Country Code  Country of Respondents Country/Region       Create Date  \
0           NaN                     NaN  united states  2021-01-12 22:35   
1           NaN                     NaN        myanmar  2021-01-12 22:35   
2           NaN                     NaN       thailand  

In [1124]:

print(hs_countries_df.shape)

(85, 26)
